## Clustering Counties Within a State Based on Industry Characteristics

Four WD types:

- Residential
- Highway
- Building
- Heavy

QCEW provides the following useful characteristics by industry and county (this is not an exhaustive list):

- Average annual number of establishments
- Average annual employment
- Average weekly wage
- Annual contributions 

We can map NAICS to WD type as follows:

- **Residential:** NAICS 2361 Residential Building Construction
- **Highway:** NAICS 2373 Highway, Street, and Bridge Construction
- **Building:** NAICS 2362 Nonresidential Building Construction
- **Heavy:** NAICS 2379 Other Heavy Construction

With this simple mapping, counties can easily be compared to one another based on industry characteristics. 

In [3]:
using CSV
using Clustering
using DataFrames
using DataFramesMeta
using Distances
using Interact
using StatsBase
using URIParser
using VegaLite
using WebIO

#### Create generic function(s) that will return graph for any industry/state

In [4]:
const industries = Dict(
    2361 => "2018.annual 2361 NAICS 2361 Residential building construction.csv",
    2373 => "2018.annual 2373 NAICS 2373 Highway, street, and bridge construction.csv",
    2362 => "2018.annual 2362 NAICS 2362 Nonresidential building construction.csv",
    2379 => "2018.annual 2379 NAICS 2379 Other heavy construction.csv"
)

function industry_df(industry::Int)
    @linq DataFrame(CSV.File("data/$(industries[industry])", normalizenames=true)) |>
    where(:agglvl_title .== "County, NAICS 4-digit -- by ownership sector") |>
    where(:own_title .== "Private")
end

industry_df (generic function with 1 method)

In [5]:
const states_abbrevs_fips = @linq DataFrame(CSV.read("data/states_abbrebs_fips.csv")) |>
    transform(fips = @. lpad(string(:fips), 2, "0"));

,state,abbrev,fips
,String,String,String
1,alabama,AL,01
2,alaska,AK,02
3,arizona,AZ,04
4,arkansas,AR,05
5,california,CA,06
6,colorado,CO,08
7,connecticut,CT,09
8,delaware,DE,10
9,district-of-columbia,DC,11


In [7]:
function create_groups_fuzzy_cmeans(state::String, industry::Int, C::Int=4, m::Float64=2.75)
        industry_data = industry_df(industry)
        df = @linq industry_data |> where(first.(:area_fips, 2) .== state)
        matrix = Matrix(hcat(
            df.annual_avg_estabs_count,
            df.annual_avg_emplvl,
            df.annual_avg_wkly_wage,
            df.annual_contributions
        )')
        matrix_normalized = StatsBase.transform(fit(ZScoreTransform, matrix, dims=2), convert.(Float64, matrix))
        weights = fuzzy_cmeans(matrix_normalized, C, m).weights
        df = DataFrame(
            fips = df.area_fips,
            county = df.area_title,
            group = [findfirst(x -> x == maximum(weights[i,:]), weights[i,:]) for i = 1:size(weights,1)]
        )
        return df
end

create_groups_fuzzy_cmeans (generic function with 3 methods)

In [8]:
function show_state_groups(state::String, industry::Int, C::Int=4, m::Float64=2.75)
    link = "https://raw.githubusercontent.com/mthelm85/topojson/master/countries/us-states/$(states_abbrevs_fips[states_abbrevs_fips.fips .== state, :abbrev][1])-$(state)-$(states_abbrevs_fips[states_abbrevs_fips.fips .== state, :state][1])-counties.json"
    return @manipulate for C = slider(2:20, value=C, label="Number of Groups"), m = slider(1.1:0.1:10.0, value=2.75, label="Fuzziness Factor")
        df = create_groups_fuzzy_cmeans(state, industry, C, m)
        @vlplot(width=1200, height=900) + 
        @vlplot(
            mark={ 
                :geoshape,
                stroke=:black
            },
            data={
                url=URI(link),
                format={
                    type=:topojson,
                    feature=Symbol("cb_2015_$(states_abbrevs_fips[states_abbrevs_fips.fips .== state, :state][1])_county_20m")
                }
            },
            transform=[
                {
                    lookup="properties.GEOID",
                    from={
                        data=df,
                        key=:fips,
                        fields=["group"]
                    }
                }
            ],
            color={
                "group:n",
                legend={title="Group"}
            },
            projection={
                typ=:naturalEarth1
            }
        ) +
        @vlplot(
            :text,
            data={
                url=URI(link),
                format={
                    type=:topojson,
                    feature=Symbol("cb_2015_$(states_abbrevs_fips[states_abbrevs_fips.fips .== state, :state][1])_county_20m")
                }
            },
                transform=[
                    {
                        calculate="geoCentroid(null, datum)",
                        as="centroid"
                    },
                    {
                        calculate="datum.centroid[0]",
                        as="centroidx"
                    },
                    {
                        calculate="datum.centroid[1]",
                        as="centroidy"
                    }
                ],
            text={field="properties.NAME", type=:nominal},
            longitude="centroidx:q",
            latitude="centroidy:q"
        )
    end
end

show_state_groups (generic function with 3 methods)

In [9]:
show_state_groups("40", 2362)

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["Number of Groups"], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:max => 20,:min => 2,:attributes => Dict{Any,Any}(:type => "range",Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient" => "horizontal"),:step => 1,:className => "slider slider is-fullwidth",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing),"index" => (Observable{Int64} with 2 listeners. Value:
4, nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "C:\\Users\\mthel\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout.js"), Asset("js", "knockout_punches", "C:\\Users\\mthel\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout_punches.js"), Asset("js", nothing, "C:\\Users\\mthel\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\all.js"), Asset("css", nothing, "C:\\Users\\mthel\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\style.css"), Asset("css", nothing, "C:\\Users\\mthel\\.julia\\packages\\Interact\\SbgIk\\src\\..\\assets\\bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set(AbstractConnection[]), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x00000000356bcb90, Task (runnable) @0x00000000356bcb90), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"2\",\"3\",\"4\",\"5\",\"6\",\"7\",\"8\",\"9\",\"10\",\"11\",\"12\",\"13\",\"14\",\"15\",\"16\",\"17\",\"18\",\"19\",\"20\"],\"changes\":WebIO.getval({\"name\":\"changes\",\